# Lab 11 - optymalizacja hiperparametrów modeli

Znaczna większość modeli posiada daotkowe zewnętrzne możliwości dostrajania za pomocą hiperparametrów. Proces poszukiwania kombinacji hiperparametrów dającej najbardziej zastysfakcjinujące wyniki nazywany jest **optymalizacją hiperparametrów**.


Najprostsze podejście do optymalizacji hiperparametrów polega na manualnym dopasowywaniu ich wartości, a nastepnie na porównywaniu wyników uzyskanych na podzbiorze testowym. Biblioteka Scikit-learn zawiera szereg klas umożliwiających automatyczną optymalizację hiperparametrów korzystając z różnych podejść.

## Metoda przeszukiwania siatki

Metoda przeszukiwania siatki to deterministyczne podejście do optymalizacji hiperparametrów polegające na testowaniu wszystkich możliwych ich kombinacji. Do realizacji procesu przeszukiwania siatki w bibliotece Scikit-learn dostępna jest klasa [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Warto zauważyć, że wszystkie klasy estymatorów posiadają metodę *fit*, co oznacza że mogą być traktowane jak tradycyjne estymatory niewymagające manualnego wskazywania hiperparametrów. Co więcej, klasy te przeprowadzają domyślnie test krzyżowy.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import Normalizer

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='pandas')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
preprocessing_pipeline = make_pipeline(
    Normalizer(),
)

In [ ]:
full_pipeline = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('classification', RandomForestClassifier(random_state=42)),
])

Metoda przeszukiwania siatki - podobnie jak inne metody optymalizacji hiperparametrów - polegają na jawnie wskazanych hiperparametrach, oraz zbiorach lub zakresach ich wartości. Dla potrzeb biblioteki *Scikit-learn* należy je wskazać jako kolekcję zawierającą słowniki z przypisanymi wartościami kombinacji do przetestowania. Każdy z hiperparametrów musi mieć również poprzedzoną nazwę za pomocą prefixu będącego nazwą etapu w potoku końcowym.

In [ ]:
hiperparam_grid = ({
      'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
      'classification__n_estimators': (16, 32, 64),
      'classification__max_features': range(4, 14, 5),
  }, {
      'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
      'classification__n_estimators': (64, 128, 256),
      'classification__max_features': ('sqrt', 'log2'),
})

In [ ]:
grid_search = GridSearchCV(full_pipeline, hiperparam_grid, cv=5, scoring='accuracy')

In [ ]:
grid_search.fit(X_train, y_train)

Za pomocą atrybutu *best_params_* można uzyskać najlepszą znalezioną kombinację hiperparametrów.

In [ ]:
grid_search.best_params_

Obiekt klasy *GridSearchCV* może zostać z powodzeniem wykorzystany jako estymator podczas wyznaczania decyzji dla danych testowych.

In [ ]:
y_pred = grid_search.predict(X_test)

Z łatwością można zauważyć, że deterministyczna metoda przeszukiwania siatki polega na sprawdzaniu wszystkich kombinacji. Dla trzech hiperparametrów, z których każdy ma zbiór trzech wartości do sprawdzenia, należy wykonać 9 cykli trening-test (zasada mnożenia). Przy 5-krotnej walidacji krzyżowej liczba cykli wzrasta do 135.

Do uzyskania bezpośredniego wyniku dla danych testowych można wykorzystać także metodę *score*.

In [ ]:
grid_search.score(X_test, y_test)

## Metoda losowego przeszukiwania siatki

W przypadku dużych przestrzeni hiperparametrów, czas ich optymalizacji może być bardzo długi. Metoda ta sprawdza się więc w przypadku niewielkich przestrzeni. W przypadku większych znacznie lepiej sprawdza się **metoda losowego przesukiwania siatki**, która polega na stochastycznym podpróbkowaniu przestrzeni hiperparametrów, co wpływa na zmniejszenie liczby cykli trening-test i - co się z tym wiąże - na skrócenie czasu trwania operacji. Do realizacji procesu losowego przeszukiwania siatki w bibliotece Scikit-learn przeznaczona jest klasa [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

In [ ]:
import math
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

Warto mieć na uwadze fakt, że w przypadku korzystania z klasy *RandomizedSearchCV* należy wskazać zakres losowego próbkowania poszczegolnych hiperparametrów zamiast zbioru ich wartości.

In [ ]:
hiperparam_distributions = {
    'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
    'classification__n_estimators': randint(16, 256),
    'classification__max_features': randint(4, int(math.sqrt(len(X[0]))))
}

In [ ]:
randomized_search = RandomizedSearchCV(
    full_pipeline,
    hiperparam_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42,
)

In [ ]:
randomized_search.fit(X_train, y_train)

Dostęp do najbardziej optymalnych hiperparametrów modelu oraz metody wyznaczania prognoz dla nowych obiektów pozostaje ten sam, co w przypadku klasy *GridSearchCV*.

In [ ]:
randomized_search.best_params_

In [ ]:
y_pred = randomized_search.predict(X_test)

In [ ]:
randomized_search.score(X_test, y_test)

## Zadania

1. Pobrać zbiórdanych *Fashion-MNIST* (https://www.openml.org/search?type=data&status=active&id=40996) za pomocą funkcji *fetch_openml*, a następnie porównać wyniki dokładności, precyzji, pełności i czułości dla hiperparametrów lasu losowego, regresji logistycznej i algorytmu k-NN za pomocą deterministycznej i stochastycznej metody przeszukiwania siatki.
2. Która z metod szybciej znalazła rozwiązanie?
3. Który z zestawów hiperparametrów zapewnił dokładniejsze wyniki klasyfikacji dla danych testowych?